In [ ]:
import os
import time
import numpy as np
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import plotly.graph_objects as go
import plotly.io as pio
import plotly
import pyvista as pv

from discretize import TensorMesh

from SimPEG.utils import plot2Ddata, surface2ind_topo, model_builder
from SimPEG.potential_fields import gravity
from SimPEG import (
    maps,
    data,
    data_misfit,
    inverse_problem,
    regularization,
    optimization,
    directives,
    inversion,
    utils,
)

t = time.time()

In [ ]:
# # Load topography
# xyz_topo = np.loadtxt(str('./Downloads/Gravity/outputs/TOPO UNGARAN UTM.txt'))

# # Load field data
# dobs = np.loadtxt(str('./Downloads/Gravity/outputs/DOBS.txt'))

# receiver_locations = dobs[:, 0:3]
# dobsx = dobs[:, -1]

# # Plot
# fig = plt.figure(figsize=(30, 16))
# n_locations = receiver_locations.shape[0]
# n_data = len(dobsx)
# v_max = np.max(np.abs(dobsx))

# ax1 = fig.add_axes([0.62, 0.15, 0.25, 0.78])
# cplot1 = plot2Ddata(
#     receiver_locations,
#     dobsx[0:n_data:1],
#     ax=ax1,
#     ncontour=30,
#     clim=(0, v_max),
#     contourOpts={"cmap": "jet"}
# )
# cplot1[0].set_clim((0, v_max))
# # ax1.set_title("$\partial g /\partial z$")
# ax1.set_title("Anomali Gravitasi")
# ax1.set_xlabel("x (m)")
# ax1.set_ylabel("y (m)")
# # ax1.set_yticks([])

# ax2 = fig.add_axes([0.89, 0.352, 0.015, 0.38])
# norm = mpl.colors.Normalize(vmin=-v_max, vmax=v_max)
# cbar = mpl.colorbar.ColorbarBase(
#     ax2, norm=norm, orientation="vertical", cmap=mpl.cm.jet
# )
# cbar.set_label("$mGal/m$", rotation=270, labelpad=16, size=12)

# plt.show()

In [ ]:
# Load topography
xyz_topo = np.loadtxt(str('./Downloads/Gravity/outputs/TOPO UNGARAN UTM.txt'))

# Load field data
dobs = np.loadtxt(str('./Downloads/Gravity/outputs/koor_upward_compressed.txt'))

# Define receiver locations and observed data
receiver_locations = dobs[:, 0:3]
dobs = dobs[:, -1]

# Plot
mpl.rcParams.update({"font.size": 12})
fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.85])
plot2Ddata(receiver_locations, dobs, ax=ax1, ncontour=40, contourOpts={"cmap": "jet"},dataloc=False, method='gaussian',shade=False)
ax1.set_title("UNGARAN Gravity Anomaly")
ax1.set_xlabel("Easting")
ax1.set_ylabel("Northing")

divider = make_axes_locatable(ax1)
cax = divider.new_vertical(size='2%', pad=0.7, pack_start = True)
ax2 = fig.add_axes(cax)
# ax2 = fig.add_axes([0.8, 0.1, 0.03, 0.85])
norm = mpl.colors.Normalize(vmin=-np.max(np.abs(dobs)), vmax=np.max(np.abs(dobs)))
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="horizontal", cmap=mpl.cm.jet
)
cbar.set_label("$mGal$", rotation=360, labelpad=5, size=12)

plt.show()

In [ ]:
print(receiver_locations)
print((dobs))


In [ ]:
topox = pv.PolyData(xyz_topo).delaunay_2d().elevation()
p = pv.Plotter()
p.add_mesh(topox)
p.enable_eye_dome_lighting()
p.show(window_size=[1024, 768])

In [ ]:
#First Horizontal Derivative
MM = np.array(receiver_locations[:,0])

fig, ax = plt.subplots(figsize = (12,4))
ax.plot(MM[4950:5050], dobs[4950:5050], '.', label='Respon y=0')
ax.tick_params(axis='both', labelsize=9)
ax.set_ylabel('Anomali Gravitasi (mGal)', fontdict={'fontsize': 9})
ax.legend(fontsize='medium', bbox_to_anchor=(0.99, 0.98), loc='upper right', borderaxespad=0)
# ax.invert_yaxis()


plt.grid()
plt.show()

In [ ]:
maximum_anomaly = np.max(np.abs(dobs))

uncertainties = 0.01 * maximum_anomaly * np.ones(np.shape(dobs))

In [ ]:
# Define the receivers. The data consists of vertical gravity anomaly measurements.
# The set of receivers must be defined as a list.
receiver_list = gravity.receivers.Point(receiver_locations, components="gz")

receiver_list = [receiver_list]

# Define the source field
source_field = gravity.sources.SourceField(receiver_list=receiver_list)

# Define the survey
survey = gravity.survey.Survey(source_field)

In [ ]:
data_object = data.Data(survey, dobs=dobs, standard_deviation=uncertainties)

In [ ]:
# dh = 600
# hx = [(dh, 5, -1.0), (dh, 20), (dh, 5, 1.0)]
# hy = [(400, 15, -1.0), (dh, 10, 1.1), (400, 15, 1.0)]
# hz = [(200, 7, -1.3), (200, 15)]
# mesh = TensorMesh([hx, hy, hz], x0 = [420377.9273, 9194952.47, -5490.093])
# fig = plt.figure(figsize=(8,12))
# ax = fig.add_subplot(111, projection='3d')
# mesh.plot_grid(ax=ax, edges=False, centers=False);
# mesh
#topo max = 2081.447
dh = 600
hx = [(dh, 5, -1.0), (dh, 20), (dh, 5, 1.0)]
hy = [(dh, 13, -1.0), (dh, 10, 1.0), (dh, 13, 1.0)]
hz = [(400, 7, -1.0), (200, 16, -1.0),(100, 21, -1.0)]
mesh = TensorMesh([hx, hy, hz], x0 = [420377.9273, 9194952.47, -6018.553])
fig = plt.figure(figsize=(8,12))
ax = fig.add_subplot(111, projection='3d')
mesh.plot_grid(ax=ax, edges=False, centers=False);
mesh

In [ ]:
# Define density contrast values for each unit in g/cc. Don't make this 0!
# Otherwise the gradient for the 1st iteration is zero and the inversion will
# not converge.
background_density = 1e-4

# Find the indices of the active cells in forward model (ones below surface)
ind_active = surface2ind_topo(mesh, xyz_topo)

# Define mapping from model to active cells
nC = int(ind_active.sum())
model_map = maps.IdentityMap(nP=nC)  # model consists of a value for each active cell
plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)

# Define and plot starting model
starting_model = background_density * np.ones(nC)
midx = int(12)
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(111)
ind_slice = midx
mesh.plot_slice(plotting_map * starting_model, normal="X", ax=ax, ind=midx, grid=True)
# ax.set_title("Model slice at y = {} m".format(mesh.cell_centers_y[ind_slice]))
plt.show()

In [ ]:
# harga = plotting_map * starting_model
# fig= go.Figure(data=go.Isosurface(
#     x= mesh.gridCC[:,0].flatten(),
#     y= mesh.gridCC[:,1].flatten(),
#     z= mesh.gridCC[:,2].flatten(),
#     value= (plotting_map * starting_model).flatten(),
#     isomin=0.00009,
#     isomax=0.00011,
#     opacity=1,
#     colorscale='tealgrn_r',
#     showscale = False
# ))
# # fig.add_annotation(
# #         x=427365,
# #         y=9202995,
# #         xref="x",
# #         yref="y",
# #         text="Gedongsongo",
# #         showarrow=False,
# #         font=dict(
# #             family="Courier New, monospace",
# #             size=16,
# #             color="#ffffff"
# #             ),
# #         align="center",
# #         arrowhead=2,
# #         arrowsize=1,
# #         arrowwidth=2,
# #         arrowcolor="#636363",
# #         ax=20,
# #         ay=-30,
# #         bordercolor="#c7c7c7",
# #         borderwidth=2,
# #         borderpad=4,
# #         bgcolor="#ff7f0e",
# #         opacity=0.8
# #         )
# fig.update_layout(
#     scene=dict(
#         xaxis=dict(type="linear"),
#         yaxis=dict(type="linear"),
#         zaxis=dict(type="linear"),
#         annotations=[
#         dict(
#             showarrow=True,
#             x=427365,
#             y=9202995,
#             z=1081,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="black",
#                 size=14),
#             text="gedongsongo",
#             textangle = 0,
#             arrowcolor = "black",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
#         dict(
#             showarrow=True,
#             x=426056,
#             y=9210013,
#             z=1081,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="black",
#                 size=14),
#             text="nglimut",
#             textangle = 0,
#             arrowcolor = "black",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
#         dict(
#             showarrow=True,
#             x=427292,
#             y=9191824,
#             z=1081,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="black",
#                 size=14),
#             text="banaran",
#             textangle = 0,
#             arrowcolor = "black",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
#         dict(
#             showarrow=True,
#             x=434395,
#             y=9201712,
#             z=1081,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="red",
#                 size=14),
#             text="1",
#             textangle = 0,
#             arrowcolor = "red",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
#         dict(
#             showarrow=True,
#             x=434219,
#             y=9200237,
#             z=1081,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="red",
#                 size=14),
#             text="2",
#             textangle = 0,
#             arrowcolor = "red",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
#         dict(
#             showarrow=True,
#             x=435363,
#             y=9202149,
#             z=1081,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="red",
#                 size=14),
#             text="3",
#             textangle = 0,
#             arrowcolor = "red",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
#         dict(
#             showarrow=True,
#             x=427058,
#             y=9203762,
#             z=1081,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="red",
#                 size=14),
#             text="4",
#             textangle = 0,
#             arrowcolor = "red",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
#         dict(
#             showarrow=True,
#             x=427046,
#             y=920,
#             z=1081,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="red",
#                 size=14),
#             text="5",
#             textangle = 0,
#             arrowcolor = "red",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
        
#         ]
#     ),
# )
# fig.show()
# # plotly.offline.plot(fig, filename='ungaran_topo.html')

In [ ]:
#Manifestasi Koordinat
XX = [434395,
434219,
435363,
427058,
427046,
426972,
426897,
427258,
427281,
426710,
437715,
437945,
426621,
437081,
428491,
426444,
427953,
430795,
427149,
425002,
427953,
430076,
424458]
YY = [9201712,
9200237,
9202149,
9203762,
9203794,
9203792,
9203790,
9203523,
9206704,
9208623,
9204683,
9204625,
9209435,
9204323,
9207266,
9208628,
9200861,
9204713,
9203392,
9199747,
9200861,
9210562,
9212923]
ZZ = [1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000,
     1000]
ann = [dict(x=x, y=y, z=z,ax=0,ay=-15,
            font=dict(
                color="red",
                size=10),
            text="⊛",
            textangle = 0,
            arrowcolor = "red",
            arrowsize = 1,
            arrowwidth = 1,
            arrowhead = 1) for x, y, z in zip(XX, YY, ZZ)]
annn = [dict(x=x, y=y, z=z,ax=0,ay=-15,
            font=dict(
                color="#D4AF37",
                size=10),
            text="⊛",
            textangle = 0,
            arrowcolor = "#D4AF37",
            arrowsize = 1,
            arrowwidth = 1,
            arrowhead = 1) for x, y, z in zip(XX, YY, ZZ)]
# fig.update_layout(
#     scene=dict(
#         xaxis=dict(type="linear"),
#         yaxis=dict(type="linear"),
#         zaxis=dict(type="log"),
#         annotations=ann
#     )
# )

In [ ]:
# harga = plotting_map * starting_model
# fig= go.Figure(data=go.Isosurface(
#     x= mesh.gridCC[:,0].flatten(),
#     y= mesh.gridCC[:,1].flatten(),
#     z= mesh.gridCC[:,2].flatten(),
#     value= (plotting_map * starting_model).flatten(),
#     isomin=0.00009,
#     isomax=0.00011,
#     opacity=1,
#     colorscale='tealgrn_r',
#     showscale = False
# ))
# fig.update_layout(
#     scene=dict(
#         xaxis=dict(type="linear"),
#         yaxis=dict(type="linear"),
#         zaxis=dict(type="linear"),
#         annotations=[
#         dict(
#             showarrow=True,
#             x=427365,
#             y=9202995,
#             z=1500,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="black",
#                 size=14),
#             text="gedongsongo",
#             textangle = 0,
#             arrowcolor = "black",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         ),
#         dict(
#             showarrow=True,
#             x=426056,
#             y=9210013,
#             z=1500,
#             ax = 0,
#             ay = -35,
#             font=dict(
#                 color="black",
#                 size=14),
#             text="nglimut",
#             textangle = 0,
#             arrowcolor = "black",
#             arrowsize = 1,
#             arrowwidth = 1,
#             arrowhead = 1
#         )
#         ]
#     ),
# )
# fig.update_layout(
#     scene=dict(
#         xaxis=dict(type="linear"),
#         yaxis=dict(type="linear"),
#         zaxis=dict(type="linear"),
#         annotations=ann
#     )
# )
# fig.show()
# # plotly.offline.plot(fig, filename='ungaran_topo.html')

In [ ]:
simulation = gravity.simulation.Simulation3DIntegral(
    survey=survey, mesh=mesh, rhoMap=model_map, actInd=ind_active
)

In [ ]:
# Define the data misfit. Here the data misfit is the L2 norm of the weighted
# residual between the observed data and the data predicted for a given model.
# Within the data misfit, the residual between predicted and observed data are
# normalized by the data's standard deviation.
dmis = data_misfit.L2DataMisfit(data=data_object, simulation=simulation)
# dmis.W = utils.sdiag(1 / uncertainties)

# Define the regularization (model objective function).
reg = regularization.Sparse(mesh, indActive=ind_active, mapping=model_map, gradientType="total")
reg.norms = np.c_[0.0, 1.0, 1.0, 1.0]
# Define how the optimization problem is solved. Here we will use a projected
# Gauss-Newton approach that employs the conjugate gradient solver.
opt = optimization.ProjectedGNCG(
    maxIter=40, lower=-0.2, upper=0.3, maxIterLS=20, maxIterCG=20, tolCG=1e-5
)

# Here we define the inverse problem that is to be solved
inv_prob = inverse_problem.BaseInvProblem(dmis, reg, opt)

# Defining a starting value for the trade-off parameter (beta) between the data
# misfit and the regularization.
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=1.0)
# Directives.Update_IRLS(norms=driver.lpnorms,  eps=driver.eps, f_min_change = 1e-4,  maxIRLSiter = 20, minGNiter = 5
# Defines the directives for the IRLS regularization. This includes setting
# the cooling schedule for the trade-off parameter.
update_IRLS = directives.Update_IRLS(
    f_min_change=1e-4,
    max_irls_iterations=20,
    coolEpsFact=1.5,
    beta_tol=1e-2,
)

# Defining the fractional decrease in beta and the number of Gauss-Newton solves
# for each beta value.
beta_schedule = directives.BetaSchedule(coolingFactor=5, coolingRate=1)


# Options for outputting recovered models and predicted data for each beta.
save_iteration = directives.SaveOutputEveryIteration(save_txt=False)

save_directive = directives.SaveModelEveryIteration()

# Updating the preconditionner if it is model dependent.
update_jacobi = directives.UpdatePreconditioner()

# Add sensitivity weights
sensitivity_weights = directives.UpdateSensitivityWeights(everyIter=False)

# The directives are defined as a list.
directives_list = [
    update_IRLS,
    sensitivity_weights,
    starting_beta,
    beta_schedule,
    save_iteration,
    save_directive,
    update_jacobi,
]

In [ ]:
# # Define the data misfit. Here the data misfit is the L2 norm of the weighted
# # residual between the observed data and the data predicted for a given model.
# # Within the data misfit, the residual between predicted and observed data are
# # normalized by the data's standard deviation.
# dmis = data_misfit.L2DataMisfit(data=data_object, simulation=simulation)

# # Define the regularization (model objective function).
# reg = regularization.Simple(mesh, indActive=ind_active, mapping=model_map)

# # Define how the optimization problem is solved. Here we will use a projected
# # Gauss-Newton approach that employs the conjugate gradient solver.
# opt = optimization.ProjectedGNCG(
#     maxIter=10, lower=-1.0, upper=1.0, maxIterLS=20, maxIterCG=10, tolCG=1e-3
# )

# # Here we define the inverse problem that is to be solved
# inv_prob = inverse_problem.BaseInvProblem(dmis, reg, opt)

# # Defining a starting value for the trade-off parameter (beta) between the data
# # misfit and the regularization.
# starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=1e1)

# # Defining the fractional decrease in beta and the number of Gauss-Newton solves
# # for each beta value.
# beta_schedule = directives.BetaSchedule(coolingFactor=5, coolingRate=1)

# # Options for outputting recovered models and predicted data for each beta.
# save_iteration = directives.SaveOutputEveryIteration(save_txt=False)
# save_directive = directives.SaveModelEveryIteration()

# # Updating the preconditionner if it is model dependent.
# update_jacobi = directives.UpdatePreconditioner()

# # Setting a stopping criteria for the inversion.
# target_misfit = directives.TargetMisfit(chifact=1)

# # Add sensitivity weights
# sensitivity_weights = directives.UpdateSensitivityWeights(everyIter=False)

# # The directives are defined as a list.
# directives_list = [
#     sensitivity_weights,
#     starting_beta,
#     beta_schedule,
#     save_iteration,
#     save_directive,
#     update_jacobi,
#     target_misfit,
# ]

In [ ]:
# Here we combine the inverse problem and the set of directives
inv = inversion.BaseInversion(inv_prob, directives_list)

# Run inversion
recovered_model = inv.run(starting_model)

In [ ]:
hasil = plotting_map * recovered_model
hasill2 = plotting_map * inv_prob.l2model

In [ ]:
# Get index of the center
midx = int(mesh.nCx / 2)
print(midx)
print(mesh.nCx)
vmin = min(hasil)
vmax = max(hasil)
print(max(hasil))
print(min(hasil))

In [ ]:
CX = int(mesh.nCx)
CY = int(mesh.nCy)
CZ = int(mesh.nCz)

In [ ]:
aaa= mesh.gridCC[:,0]
bbb= mesh.gridCC[:,1]
ccc= mesh.gridCC[:,2]

print(aaa,bbb,ccc)

In [ ]:

# hasil[hasil == -100] = np.nan
fig = go.Figure()
# zone1= go.Isosurface(
#     x= mesh.gridCC[:,0],
#     y= mesh.gridCC[:,1],
#     z= mesh.gridCC[:,2],
#     value= hasil.flatten(),
#     isomin=0.001,
#     isomax=0.3,
#     opacity=1,
#     colorscale='blues',
#     showscale= False
# )
# zone1= go.Isosurface(
#     x= mesh.gridCC[:,0],
#     y= mesh.gridCC[:,1],
#     z= mesh.gridCC[:,2],
#     value= hasil,
#     isomin=0.201,
#     isomax=0.3,
#     opacity=0.8,
#     colorscale='reds_r',
#     showscale= False
# )
zone2= go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= hasil,
    isomin=-0.2,
    isomax=-0.101,
    surface_count = 5,
    opacity=1,
    colorscale='reds_r',
    colorbar_x=-0.2,
    colorbar_xpad=10,
    colorbar_ypad=10,
    showscale= True
)
zone3= go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= hasil,
    isomin=-0.1,
    isomax=-0.001,
    surface_count = 3,
    surface=dict(show=True,count=1, fill=0.8),
    caps=go.isosurface.Caps(
        z=dict(show=False),
        x=dict(show=False),
        y=dict(show=False)),
    opacity=0.8,
    colorscale='oranges_r',
    colorbar_x=-0.1,
    colorbar_xpad=10,
    colorbar_ypad=10,
    showscale= True
)
# zone1= go.Isosurface(
#     x= mesh.gridCC[:,0],
#     y= mesh.gridCC[:,1],
#     z= mesh.gridCC[:,2],
#     value= hasil,
#     isomin=-0.001,
#     isomax=-0.1,
#     colorscale='greens',
#     opacity = 0.8,
#     showscale= False
# )
zone4= go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= hasil,
    isomin=0.001,
    isomax=0.1,
    surface_count = 10,
    surface=dict(show=True,count=1, fill=0.8),
    caps=go.isosurface.Caps(
        z=dict(show=False),
        x=dict(show=False),
        y=dict(show=False)),
    opacity=0.8,
    colorscale='aggrnyl_r',
    colorbar_xpad=1,
    colorbar_ypad=1,
#     reversescale=True,
    colorbar_orientation="h",
#     colorbar_x=-0,
    showscale= True
)
zone5= go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= hasil,
    isomin=0.101,
    isomax=0.2,
    opacity=1,
    colorscale='blues_r',
    colorbar_x=1,
    colorbar_xpad=10,
    colorbar_ypad=10,
    showscale= True
)
zone6= go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= hasil,
    isomin=0.201,
    isomax=0.3,
    opacity=1,
    colorscale='purp_r',
    colorbar_x=1.1,
    colorbar_xpad=10,
    colorbar_ypad=10,
    showscale= True
)
# 
# fig.add_trace(zone1)
fig.add_trace(zone2)
fig.add_trace(zone3)
fig.add_trace(zone4)
fig.add_trace(zone5)
fig.add_trace(zone6)
fig.update_layout(
    scene=dict(
        xaxis=dict(type="linear"),
        yaxis=dict(type="linear"),
        zaxis=dict(type="linear"),
        annotations=[
        dict(
            showarrow=True,
            x=427365,
            y=9202995,
            z=1500,
            ax = 0,
            ay = -35,
            
            font=dict(
                color="black",
                size=14),
            text="gedongsongo",
            textangle = 0,
            arrowcolor = "black",
            arrowsize = 1,
            arrowwidth = 1,
            arrowhead = 1
        ),
        dict(
            showarrow=True,
            x=426056,
            y=9210013,
            z=1500,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=14),
            text="nglimut",
            textangle = 0,
            arrowcolor = "black",
            arrowsize = 1,
            arrowwidth = 1,
            arrowhead = 1
        ),
        dict(
            showarrow=False,
            x=429720,
            y=9216600,
            z=-6000,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=18),
            text="N",
            textangle = 0
        ),    
        dict(
            showarrow=False,
            x=429720,
            y=9195000,
            z=-6000,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=18),
            text="S",
            textangle = 0
        ),
        dict(
            showarrow=False,
            x=420000,
            y=9205752,
            z=-6000,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=18),
            text="W",
            textangle = 0
        ),
        dict(
            showarrow=False,
            x=438500,
            y=9205752,
            z=-6000,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=18),
            text="E",
            textangle = 0
        )
        ]
    ),
)
fig.update_layout(
    scene=dict(
        xaxis=dict(type="linear"),
        yaxis=dict(type="linear"),
        zaxis=dict(type="linear"),
        annotations=ann
    )
)
# fig.show()
# pio.write_image(fig, 'filename.pdf', width=1920, height=1080)
plotly.offline.plot(fig, filename='ungaran_variasi2.html')

In [ ]:
fig = go.Figure()
# zone1= go.Isosurface(
#     x= mesh.gridCC[:,0],
#     y= mesh.gridCC[:,1],
#     z= mesh.gridCC[:,2],
#     value= hasil.flatten(),
#     isomin=0.001,
#     isomax=0.3,
#     opacity=1,
#     colorscale='blues',
#     showscale= False
# )
zonex1= go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= hasil,
    isomin=-0.201,
    isomax=-0.001,
    opacity=1,
    colorscale='mint',
    colorbar_x = -0.1,
#     colorbar_xpad=1,
#     colorbar_ypad=1,
    showscale= True
)
zonex2= go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= hasil,
    isomin=0.001,
    isomax=0.301,
    opacity=0.8,
    colorscale='reds_r',
    colorbar_x = 1,
    showscale= True
)
fig.add_trace(zonex1)
fig.add_trace(zonex2)
fig.update_layout(
    scene=dict(
        xaxis=dict(type="linear"),
        yaxis=dict(type="linear"),
        zaxis=dict(type="linear"),
        annotations=[
        dict(
            showarrow=True,
            x=427365,
            y=9202995,
            z=1500,
            ax = 0,
            ay = -35,
            
            font=dict(
                color="black",
                size=14),
            text="gedongsongo",
            textangle = 0,
            arrowcolor = "black",
            arrowsize = 1,
            arrowwidth = 1,
            arrowhead = 1
        ),
        dict(
            showarrow=True,
            x=426056,
            y=9210013,
            z=1500,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=14),
            text="nglimut",
            textangle = 0,
            arrowcolor = "black",
            arrowsize = 1,
            arrowwidth = 1,
            arrowhead = 1
        ),
        dict(
            showarrow=False,
            x=429720,
            y=9216600,
            z=-6000,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=18),
            text="N",
            textangle = 0
        ),    
        dict(
            showarrow=False,
            x=429720,
            y=9195000,
            z=-6000,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=18),
            text="S",
            textangle = 0
        ),
        dict(
            showarrow=False,
            x=420000,
            y=9205752,
            z=-6000,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=18),
            text="W",
            textangle = 0
        ),
        dict(
            showarrow=False,
            x=438500,
            y=9205752,
            z=-6000,
            ax = 0,
            ay = -35,
            font=dict(
                color="black",
                size=18),
            text="E",
            textangle = 0
        )    
        ]
    ),
)
fig.update_layout(
    scene=dict(
        xaxis=dict(type="linear"),
        yaxis=dict(type="linear"),
        zaxis=dict(type="linear"),
        annotations=annn)
)
    
plotly.offline.plot(fig, filename='ungaran_tinggirendah.html')

In [ ]:
fig = go.Figure()
# zone1= go.Isosurface(
#     x= mesh.gridCC[:,0],
#     y= mesh.gridCC[:,1],
#     z= mesh.gridCC[:,2],
#     value= hasil.flatten(),
#     isomin=0.001,
#     isomax=0.3,
#     opacity=1,
#     colorscale='blues',
#     showscale= False
# )
zoney1= go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= hasil,
    isomin=-0.199,
    isomax=0.299,
    surface_count = 20,
    surface=dict(show=True,count=1, fill=0.8),
    caps=go.isosurface.Caps(
        z=dict(show=False),
        x=dict(show=False),
        y=dict(show=False)),
    opacity=0.8,
    colorscale='aggrnyl_r',
    showscale= False
)
fig.add_trace(zoney1)
plotly.offline.plot(fig, filename='ungaran_blues.html')

## Density 0 with grid

In [ ]:
# Plot Recovered Model X
for i in range (1,CX,1):
    fig = plt.figure(figsize=(10, 5))
#     plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)
#     hasil = plotting_map * recovered_model
    hasil[hasil == -100] = np.nan
    ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
    mesh.plotSlice(
        plotting_map * recovered_model,
        normal="X",
        ax=ax1,
        ind=i,
        grid=True,
        clim=(-0.2, 0.3),
        pcolorOpts={"cmap": "jet"},
    )
    ax1.set_title("Slice = {} ".format(i))

    # ax2 = fig.add_axes([0.85, 0.1, 0.05, 0.8])
    divider = make_axes_locatable(ax1)
    cax = divider.new_vertical(size='5%', pad=0.7, pack_start = True)
    ax2 = fig.add_axes(cax)
    norm = mpl.colors.Normalize(vmin=-0.2, vmax=0.3)
    cbar = mpl.colorbar.ColorbarBase(
        ax2, norm=norm, orientation="horizontal", cmap=mpl.cm.jet
    )
    cbar.set_label("$g/cm^3$", rotation=360, labelpad=0, size=12)
#     plt.savefig('XG_{}.png'.format(i))
#     ax1.invert_xaxis()
    plt.show()

In [ ]:
# Plot Recovered Model L2
for i in range (1,CX,1):
    fig = plt.figure(figsize=(10, 5))
#     plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)
#     hasil = plotting_map * recovered_model
    hasill2[hasill2 == -100] = np.nan
    ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
    mesh.plotSlice(
        hasill2,
        normal="X",
        ax=ax1,
        ind=i,
        grid=True,
        clim=(-0.3, 0.3),
        pcolorOpts={"cmap": "jet"},
    )
    ax1.set_title("Potongan model ke = {} ".format(i))

    # ax2 = fig.add_axes([0.85, 0.1, 0.05, 0.8])
    divider = make_axes_locatable(ax1)
    cax = divider.new_vertical(size='5%', pad=0.7, pack_start = True)
    ax2 = fig.add_axes(cax)
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = mpl.colorbar.ColorbarBase(
        ax2, norm=norm, orientation="horizontal", cmap=mpl.cm.jet
    )
    cbar.set_label("$g/cm^3$", rotation=360, labelpad=0, size=12)
    plt.savefig('XG_{}.png'.format(i))
#     ax1.invert_xaxis()
    plt.show()

## Density 0 without grid

In [ ]:
# Plot Recovered Model
for i in range (1,CZ,1):
    fig = plt.figure(figsize=(10, 5))
    plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)
    hasil = plotting_map * recovered_model
    hasil[hasil == -100] = np.nan
    ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
    mesh.plotSlice(
        plotting_map * recovered_model,
        normal="Z",
        ax=ax1,
        ind=i,
        grid=False,
        clim=(-0.3, 0.3),
        pcolorOpts={"cmap": "jet"},
    )
    ax1.set_title("Potongan model ke = {} ".format(i))

    # ax2 = fig.add_axes([0.85, 0.1, 0.05, 0.8])
    divider = make_axes_locatable(ax1)
    cax = divider.new_vertical(size='5%', pad=0.7, pack_start = True)
    ax2 = fig.add_axes(cax)
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = mpl.colorbar.ColorbarBase(
        ax2, norm=norm, orientation="horizontal", cmap=mpl.cm.jet
    )
    cbar.set_label("$g/cm^3$", rotation=360, labelpad=0, size=12)
    plt.savefig('Z_{}.png'.format(i))
#     ax1.invert_xaxis()
    plt.show()

## Density 2,69 with grid

In [ ]:
# Plot Recovered Model
for i in range (1,CX,1):
    fig = plt.figure(figsize=(10, 5))
    plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)
    hasil = (plotting_map * recovered_model) + 2.69
    hasil[hasil == -100] = np.nan
    ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
    mesh.plotSlice(
        hasil,
        normal="X",
        ax=ax1,
        ind=i,
        grid=True,
        clim=(2.39, 2.99),
        pcolorOpts={"cmap": "jet"},
    )
    ax1.set_title("Potongan model ke = {} ".format(i))

    # ax2 = fig.add_axes([0.85, 0.1, 0.05, 0.8])
    divider = make_axes_locatable(ax1)
    cax = divider.new_vertical(size='5%', pad=0.7, pack_start = True)
    ax2 = fig.add_axes(cax)
    norm = mpl.colors.Normalize(vmin=2.39, vmax=2.99)
    cbar = mpl.colorbar.ColorbarBase(
        ax2, norm=norm, orientation="horizontal", cmap=mpl.cm.jet
    )
    cbar.set_label("$g/cm^3$", rotation=360, labelpad=0, size=12)
    plt.savefig('X269G_{}.png'.format(i))
#     ax1.invert_xaxis()
    plt.show()

## Density 2,69 without grid

In [ ]:
# Plot Recovered Model
for i in range (1,CZ,1):
    fig = plt.figure(figsize=(10, 5))
    plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)
    hasil = (plotting_map * recovered_model) + 2.69
    hasil[hasil == -100] = np.nan
    ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
    mesh.plotSlice(
        hasil,
        normal="Z",
        ax=ax1,
        ind=i,
        grid=False,
        clim=(2.39, 2.99),
        pcolorOpts={"cmap": "jet"},
    )
    ax1.set_title("Potongan model ke = {} ".format(i))

    # ax2 = fig.add_axes([0.85, 0.1, 0.05, 0.8])
    divider = make_axes_locatable(ax1)
    cax = divider.new_vertical(size='5%', pad=0.7, pack_start = True)
    ax2 = fig.add_axes(cax)
    norm = mpl.colors.Normalize(vmin=2.39, vmax=2.99)
    cbar = mpl.colorbar.ColorbarBase(
        ax2, norm=norm, orientation="horizontal", cmap=mpl.cm.jet
    )
    cbar.set_label("$g/cm^3$", rotation=360, labelpad=0, size=12)
    plt.savefig('Z269_{}.png'.format(i))
#     ax1.invert_xaxis()
    plt.show()

In [ ]:
#Hi, I'm using plotly to see the 3D isometric of my gravity inversion result. But I can't see the topography. I'm using (value[value == -100] = np.nan). It works for showing the topography on  matplotlib but not in plotly. Does anyone know what should I do?
print(len(recovered_model))
print(len(receiver_locations))
print(np.max(recovered_model))
print(np.min(recovered_model))

In [ ]:
hasil = plotting_map * recovered_model
hasil[hasil == -100] = np.nan
fig= go.Figure(data=go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= (hasil),
    isomin=0.001,
    isomax=0.3,
    opacity=1,
    colorscale='jet'
))
fig.show()
# # pio.write_image(fig, 'filename.pdf', width=1920, height=1080)
plotly.offline.plot(fig, filename='ungaran_tinggi.html')

In [ ]:
hasil = plotting_map * recovered_model
hasil[hasil == -100] = np.nan
fig= go.Figure(data=go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= (hasil),
    isomin=-0.3,
    isomax=-0.001,
    opacity=1,
    colorscale='jet'
))

fig.show()
# pio.write_image(fig, 'filename.pdf', width=1920, height=1080)
plotly.offline.plot(fig, filename='ungaran_rendah.html')

In [ ]:
dtrue = inv_prob.dpred

In [ ]:
#First Horizontal Derivative
MM = np.array(receiver_locations[1:10000:100,1])

fig, ax = plt.subplots(figsize = (12,5))
ax.plot(MM, dobs[55:10000:100], 'o', label='Data')
ax.plot(MM, dtrue[55:10000:100], '-', label='Model')
ax.tick_params(axis='both', labelsize=9)
ax.set_ylabel('Gravity Anomaly (mGal)', fontdict={'fontsize': 12})
ax.set_xlabel('Coordinate (UTM)', fontdict={'fontsize': 12})
ax.legend(fontsize='medium', bbox_to_anchor=(0.99, 0.98), loc='upper right', borderaxespad=0)
# ax.invert_yaxis()

plt.grid()
plt.xlim(9194952, 9215000)
plt.show()

In [ ]:
dp = (dobs - dtrue)**2
dpx = sum(dp)/len(dobs)
print("\nLp model final misfit:" + str(np.sum(((dobs-dtrue)/uncertainties)**2.)))
print("Lp misfit sum(obs-calc)/nobs: %.3f mGal"  %np.divide(np.sum(np.abs(dobs-dtrue)), len(dobs)))
print("Lp RMS misfit: %.3f mGal" %np.sqrt(np.divide(np.sum((dobs-dtrue)**2),len(dobs))))

In [ ]:
from plotly.subplots import make_subplots
trace1 = go.Figure(data=go.Isosurface(
    x = mesh.gridCC[:,0],
    y = mesh.gridCC[:,1],
    z = mesh.gridCC[:,2],
    value= (hasil),
    isomin=-0.3,
    isomax=-0.001,
    opacity=1,
    colorscale='jet'
))
trace2 = go.Figure(data=go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= (hasil),
    isomin=0.001,
    isomax=0.3,
    opacity=1,
    colorscale='jet'
))
figx = make_subplots(specs=[[{"secondary_y":True}]])
figx.add_trace(trace1)
figx.add_trace(trace2, secondary_y=True)
iplot(figx)

In [ ]:
f = plotting_map * recovered_model
X, Y, Z = mesh.gridCC[:2]
vals = f(X,Y,Z)

In [ ]:
go.Figure(data=go.Isosurface(
    x= mesh.gridCC[:,0],
    y= mesh.gridCC[:,1],
    z= mesh.gridCC[:,2],
    value= (hasil),
    isomin=0.001,
    isomax=0.3,
    opacity=1,
    showscale=False,
    colorscale='jet'
))